<a href="https://colab.research.google.com/github/Sxbxn/Furry_Friends/blob/ai_2/GANomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -qq /content/drive/MyDrive/teamproject/Ch04.zip

replace 0/test_img/test/D_61_20120525_IF_0029_NOR_Ch04_20211214_2907.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [14]:
def make_encoder_model():
    inputs = layers.Input(shape=(224, 224, 3))
    x = layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(inputs)
    x = layers.Conv2D(64, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    outputs = layers.Dense(32, activation='relu')(x)
    return Model(inputs=inputs, outputs=outputs)


In [15]:
def make_decoder_model():
    inputs = layers.Input(shape=(32,))
    x = layers.Dense(1024, activation='relu')(inputs)
    x = layers.Dense(7*7*512, activation='relu')(x)
    x = layers.Reshape((7, 7, 512))(x)
    x = layers.Conv2DTranspose(512, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(512, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(256, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(256, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(128, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(128, 3, strides=1, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(64, 3, strides=1, padding='same', activation='relu')(x)
    outputs = layers.Conv2DTranspose(3, 3, strides=2, padding='same', activation='sigmoid')(x)
    return Model(inputs=inputs, outputs=outputs)


In [16]:
# Define the discriminator network
def make_discriminator_model():
    inputs = layers.Input(shape=(32,))
    x = layers.Dense(128, activation='relu')(inputs)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return keras.models.Model(inputs=inputs, outputs=outputs)

In [17]:
class GANomaly(keras.models.Model):
  def __init__(self):
    super(GANomaly, self).__init__()
    self.encoder = make_encoder_model()
    self.decoder = make_decoder_model()
    self.discriminator = make_discriminator_model()

  def call(self, x): 
      z = self.encoder(x)  # latent vector.
      x_hat = self.decoder(z)
      d = self.discriminator(z)
      return x_hat, d

In [18]:
# Define the losses
adversarial_loss = keras.losses.BinaryCrossentropy(from_logits=False)
reconstruction_loss = keras.losses.MeanAbsoluteError()

# Define the optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Define the batch size and number of epochs
batch_size = 32
num_epochs = 100
image_size = (224, 224)

In [19]:
# image_paths = tf.data.Dataset.list_files('/content/0/train_img')

# # Define a function to load and preprocess the images
# def load_and_preprocess_image(file_path):
#     # Load the image
#     image = tf.io.read_file(file_path)
#     image = tf.image.decode_jpeg(image, channels=3)
#     # Resize and normalize the image
#     image = tf.image.resize(image, image_size)
#     image = image / 255.
#     return image

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define a data generator to load and preprocess image data
data_generator = ImageDataGenerator(rescale = 1./255)

# Load training data from a directory and preprocess it using the data generator
train_dataset = data_generator.flow_from_directory('/content/0/train_img', batch_size=32, target_size = image_size)

Found 6184 images belonging to 1 classes.


In [20]:
ganomaly = GANomaly() # create the GANomaly model

In [41]:
import numpy as np

for epoch in range(num_epochs):
    for i, batch in enumerate(train_dataset):
        images = batch[0]
        labels = np.zeros((batch_size, 1))
        labels[:batch_size//2] = 1
        labels = tf.cast(labels,tf.float32)

        # Train discriminator
        with tf.GradientTape() as tape:
            generated_image, original_latent = ganomaly(images)
            _, generated_latent = ganomaly(generated)
            d_loss_real = adversarial_loss(original_latent, labels)
            d_loss_fake = adversarial_loss(generated_latent, tf.zeros((batch_size, 1)))
            # print(d_loss_real,d_loss_fake)
            d_loss_reconstructed = reconstruction_loss(images, generated_image)
            # print(d_loss_reconstructed)
            d_loss = d_loss_real + d_loss_fake + d_loss_reconstructed
        grads = tape.gradient(d_loss, ganomaly.trainable_weights)
        optimizer.apply_gradients(zip(grads, ganomaly.trainable_weights))

        # Train generator
        with tf.GradientTape() as tape:
            generated, original_latent = ganomaly(images)
            g_loss = adversarial_loss(original_latent, labels) + reconstruction_loss(images, generated)
        grads = tape.gradient(g_loss, ganomaly.trainable_weights)
        optimizer.apply_gradients(zip(grads, ganomaly.trainable_weights))

        if i % 100 == 0:
            print(f'Epoch {epoch+1}, Batch {i+1}, D Loss: {d_loss.numpy():.4f}, G Loss: {g_loss.numpy():.4f}')

Epoch 1, Batch 1, D Loss: 7.8096, G Loss: 7.8088
Epoch 1, Batch 101, D Loss: 7.7709, G Loss: 7.7722


InvalidArgumentError: ignored

In [33]:
ganomaly.discriminator.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_10 (Dense)            (None, 128)               4224      
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 4,353
Trainable params: 4,353
Non-trainable params: 0
_________________________________________________________________
